In [1]:
import socket
import pickle
import select
import time
import sys

class ProxyError(OSError):
    pass

class ProxyTimeout(ProxyError):
    pass


class BaseProtocol():
    """ Mixin / common code for client & server side 
    protocols. 
    """
    SOCK_TIMEOUT = 30
    RECV_TIMEOUT = 30
    
    def __init__(self):
        self._queue = {}
        
    def _makefiles(self):
        self._wfile = self._sock.makefile("wb")
        self._rfile = self._sock.makefile("rb")
        self._sock.settimeout(self.SOCK_TIMEOUT)
    
    def close(self):
        if self._wfile:
            self._wfile.flush()
            self._wfile.close()
            self._wfile = None
        if self._rfile:
            self._rfile.close()
            self._rfile = None
            
        self._sock.shutdown(socket.SDRW)
        self._sock.close()
        self._sock = None
        
    def send(self, data):
        pickle.dump(data, self._wfile)
        self._wfile.flush()
        
    def recv_one_msg(self):
        r, w, x = select.select([self._sock], [], [], self.SOCK_TIMEOUT)
        if r:
            data = msg_load(self._rfile)
            if data.msg == "INTERNAL_ERROR":
                raise ProxyError("Internal error: '%s'"%data)
            return data
        else:
            raise ProxyTimeout("Socket read timeout")
        
    def recv(self, id, timeout=-1, hard_timeout=None):
        data = self._queue.pop(id, None)
        if data is not None:
            return data
        
        if timeout < 0:
            timeout = self.RECV_TIMEOUT
        
        end = time.time() + timeout
        
        if hard_timeout is not None:
            hard_end = max(end, time.time() + hard_timeout)
        else:
            hard_end = -1
            
        while True:
            data = self.recv_one_msg()
            if data.msg == "HEARTBEAT":
                end = time.time() + timeout
                if hard_end > 0 and end > hard_end:
                    end = hard_end
                continue
                
            if data.id != id:
                self._queue[data.id] = data
            else:
                return data

            if time.time() > end:
                raise ProxyTimeout("Took too long to get confirmation for result")

        
class MessageData():
    def __init__(self):
        self.id = None
        self.msg = None
        self.data = None

In [2]:
from scripts.tools.issuetracker import clientapi as capi
import importlib
capi = importlib.reload(capi)
import threading
import datetime

class RedmineProxyError(Exception):
    pass



class RedmineCache():
    def __init__(self):
        self._cache = {}
        self._updated = datetime.datetime(1970, 1, 1)
        
    def clear(self):
        self._cache.clear()
        
    def lookup(self, id):
        return self._cache[id]
    
    def get_all(self):
        return self._cache.copy()
    
    def update(self, issues):
        for iss in issues:
            self._cache[iss.id] = iss
        
    @property
    def last_update(self):
        return self._updated
    
    def filter(self, pred):
        out = {}
        for iid, iss in self._cache.items():
            try:
                res = pred(iss)
            except Exception:
                raise ValueError("Exception occurred while performing filter with custom predicate")
            out[iid] = iss
        return out
    
    def filter2(self, filters):
        out = {}
        for iid, iss in self._cache.items():
            b = True
            for v1, op, v2 in filters:
                v1 = getattr(iss, v1)
                if op == "==":
                    b = b and v1 == v2
                elif op == "!=":
                    b = b and v1 != v2
                elif op == ">=":
                    b = b and v1 >= v2
                elif op == "<=":
                    b = b and v1 <= v2
                elif op == ">":
                    b = b and v1 > v2
                elif op == "<":
                    b = b and v1 < v2
                else:
                    raise ValueError(op)
            if b:
                out[iid] = iss
        return out
    
    def set_update_time(self, time=None):
        if time is None:
            self._updated = datetime.datetime.now()
        else:
            assert isinstance(time, datetime.datetime)
            self._updated = time
        
class _StopManagerThread(Exception):
    pass

class RedmineCacheManager(threading.Thread):
    def __init__(self, update_interval=30):
        self._api = None
        self.lock = threading.Lock()  # just in case
        self._stop = threading.Event()
        self._cache = RedmineCache()
        self._update_interval = update_interval # seconds
        self._initialized = False
        super().__init__(daemon=True)
        
    @property
    def api(self):
        if self._api is None:
            self._api = capi.IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        return self._api
        
    def run(self):
        self._stop.clear()
        try:
            self._run()
        except _StopManagerThread:
            pass
        
    @property
    def is_initialized(self):
        return self._initialized
        
    def _run(self):
        # on first iteration, populate cache with no delay
        wait = 0
        while True:
            updated_on = ">=" + self._cache.last_update.isoformat()
            iiter = self._do_download(updated_on)
            self._update_from_iter(iiter, 0)
            self._cache.set_update_time()
            
            if self._stop.wait(self._update_interval):
                raise _StopManagerThread()
                
            wait = 1
            self._initialized = True
            
    def _update_from_iter(self, iiter, wait):
        for issues in iiter:
            self._cache.update(issues)
            if self._stop.wait(wait):
                raise _StopManagerThread()
    
    def _update_issues(self, issues):
        with self.lock:
            self._cache.update(issues)

    def _do_download(self, updated_on):
        try:
            return self.api.download_issues2(modified_on=updated_on, status_id="*")
        except Exception as e:
            raise RedmineProxyError("Error occurred collecting issues: \"%s\""%str(e))
        
    def stop(self):
        self._stop.set()
        
    def get_all(self):
        with self.lock:
            return self._cache.get_all()
    
    def lookup(self, id):
        with self.lock:
            return self._cache.lookup(id)
    
    def filter(self, filters):
        with self.lock:
            return self._cache.filter2(filters)

In [3]:
import socketserver
import http.server
import urllib
import base64
import pickle
import http
import socket

def encode(item):
    b = pickle.dumps(item)
    return base64.b64encode(b)

def decode(string):
    b = base64.b64decode(string)
    return pickle.loads(b)

def parse_params(params):
    out = {}
    for k, v in params.items():
        if len(v) > 1:
            raise ValueError("Too many arguments for %r: %r" % (k, ", ".join(v)))
        out[k] = v[0]
    return out

class RedmineRequestHandler(http.server.SimpleHTTPRequestHandler):
    default_request_version = "HTTP/1.1"
    
    def do_GET(self):
        res = urllib.parse.urlparse(self.path) 
        qs = res.query
        params = urllib.parse.parse_qs(qs)
        params = parse_params(params)
        if res.path == "/cache":
            if not self.server.manager.is_initialized:
                return self.send_error(http.HTTPStatus.BAD_REQUEST, "Service is still initializing")
            self._do_cache_request(params)
        else:
            self.send_error(http.HTTPStatus.BAD_REQUEST, "Unknown path: %r" % res.path)
    
    def _do_cache_request(self, params):
    
        if "issues" in params:
            ireq = params["issues"]
            if ireq == 'all':
                try:
                    issues = self._do_request_all_issues()
                except Exception as e:
                    return self.send_error(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Big Oops: %r"%str(e))
            else:
                l = ireq.split(",")
                try:
                    issues = list(map(int, l))
                except ValueError:
                    return self.send_error("invalid value for 'issues': %r"%issues)
                try:
                    ret = [self.server.manager.lookup(i) for i in issues]
                except KeyError as e:
                    return self.send_error(http.HTTPStatus.NOT_FOUND, "failed to find key: '%'" % str(e))
        elif "filters" in params:
            filters = decode(params["filters"])
            try:
                issues = self.server.manager.filter(filters)
            except ValueError:
                return self.send_error(http.HTTPStatus.BAD_REQUEST, "bad filter string: %r"%filters)
        else:
            return self.send_error(http.HTTPStatus.BAD_REQUEST, "Unknown args: %r" % params)
            
        rsp = encode(issues)
        self.send_response(200)
        self.send_header("Content-type", "text/plain")
        self.send_header("Content-Length", str(len(rsp)))
        self.end_headers()
        self.wfile.write(rsp)
        self.wfile.flush()
            
    def _do_request_all_issues(self):
        return self.server.manager.get_all()

class RedmineServer(socketserver.ThreadingTCPServer):
    
    def __init__(self, addr, manager=None):
        self.manager = manager or RedmineCacheManager()
        self.manager.start()
        super().__init__(addr, RedmineRequestHandler)
    
    # from http.server.HTTPServer
    def server_bind(self):
        """Override server_bind to store the server name."""
        socketserver.TCPServer.server_bind(self)
        host, port = self.socket.getsockname()[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
        
    def serve_forever(self, interval=0.05):
        threading.Thread(target=super().serve_forever, args=(interval,), daemon=True).start()
    

In [4]:
try:
    manager
except NameError:
    manager = RedmineCacheManager()
rs = RedmineServer(("localhost", 11649), manager)
rs.serve_forever(0.00)

127.0.0.1 - - [21/Mar/2019 18:34:04] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2019 18:34:05] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -


127.0.0.1 - - [21/Mar/2019 18:34:06] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2019 18:34:07] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -


127.0.0.1 - - [21/Mar/2019 18:34:08] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -


127.0.0.1 - - [21/Mar/2019 18:34:09] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2019 18:34:10] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -


127.0.0.1 - - [21/Mar/2019 18:34:11] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -


127.0.0.1 - - [21/Mar/2019 18:34:12] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -


127.0.0.1 - - [21/Mar/2019 18:34:13] "GET /cache?filters=gANdcQBYEAAAAHNwcmludF9taWxlc3RvbmVxAVgCAAAAPT1xAlgDAAAAMy4wcQOHcQRhLg== HTTP/1.1" 200 -


In [ ]:
import requests

def decode(string):
    b = base64.b64decode(string)
    return pickle.loads(b)

class RedmineClient():
    _url = "http://localhost:11649/"
    def __init__(self):
        self.session = requests.Session()
    def get_all(self):
        r = self.session.get(self._url + "cache" + "?issues=all")
        r.raise_for_status()
        return decode(r.content.decode())

In [ ]:
# rs.shutdown()
# rs.manager.stop()
# rs.socket.close()

In [ ]:
# capi = importlib.reload(capi)
# api = capi.IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')

# # url = capi.uj(api._base_url, api._issues_url + ".json")
# # now = datetime.datetime.now()
# # params = {"updated_on": ">=" + now.isoformat()}
# # print(now)
# # url += "?" + urllib.parse.urlencode(params)
# # r = api._sess.get(url, auth=api._auth)

# # url

# # r.content

# _=next(api.download_issues())